# Maximizing Profit: Targeted Restocking and VIP Customer Engagement for Sustainable Growth

## Introduction

The goal of this analysis is to evaluate product inventory and customer behavior to guide restocking decisions, improve customer engagement, and assess how much can be invested in acquiring new customers. By examining product sales performance, customer profitability, and customer acquisition trends, we aim to derive actionable insights for store management.

The approach includes three main analyses: identifying which products need restocking based on stock levels and sales performance, categorizing customers into VIP and less-engaged groups based on profit contributions, and calculating the Customer Lifetime Value (LTV) to inform marketing spend on new customers. We used SQL queries to extract relevant data from the store's database and applied logical calculations to arrive at these insights.

The results revealed that certain high-demand products like the **1960 BSA Gold Star DBD34** require immediate restocking, while VIP customers such as **Diego Freyre** in **Madrid** contribute significantly to the store's profit. Additionally, the computed **LTV** of **$39,039.59** suggests the store can allocate a substantial budget to customer acquisition, particularly as the rate of new customer arrivals has been declining.

## Table of Contents
- [Database Schema for Scale Model Cars Store](#Database-Schema-for-Scale-Model-Cars-Store)
- [Loading the Database and Exploring its tables](#Loading-the-Database-and-Exploring-its-tables)
- [Analysis of Product Stock Levels](#Analysis-of-Product-Stock-Levels)
- [Question 1: Which Products Should We Order More of or Less of?](#Question-1:-Which-Products-Should-We-Order-More-of-or-Less-of?)
- [Analysis of Customer Profit and Marketing Strategy](#Analysis-of-Customer-Profit-and-Marketing-Strategy)
- [Question 2: How Should We Match Marketing and Communication Strategies to Customer Behavior?](#Question-2:-How-Should-We-Match-Marketing-and-Communication-Strategies-to-Customer-Behavior?)
- [Analysis of Customer Lifetime Value](#Analysis-of-Customer-Lifetime-Value)
- [Question 3: How Much Can We Spend on Acquiring New Customers?](#Question-3:-How-Much-Can-We-Spend-on-Acquiring-New-Customers?)
- [Conclusion](#Conclusion)

## **Database Schema for Scale Model Cars Store**

### This database contains the following tables and their relationships:

---

### 1. **Table**: `customers`
**Description**: Stores customer data, including contact information, address, and associated sales representative.  
**Primary Key**: `customerNumber`  
**Foreign Key**: `salesRepEmployeeNumber` → `employees(employeeNumber)`
  - **Columns**:
    - `customerNumber`: Unique identifier for each customer.
    - `customerName`: The name of the customer.
    - `contactLastName`, `contactFirstName`: Contact person's name.
    - `phone`: Contact phone number.
    - `addressLine1`, `addressLine2`: Address of the customer.
    - `city`, `state`, `postalCode`, `country`: Location of the customer.
    - `salesRepEmployeeNumber`: Employee responsible for the customer.
    - `creditLimit`: Customer's credit limit.

---

### 2. **Table**: `employees`
**Description**: Contains employee details, including their office location and supervisor.  
**Primary Key**: `employeeNumber`  
**Foreign Key**: `officeCode` → `offices(officeCode)`, `reportsTo` → `employees(employeeNumber)`
  - **Columns**:
    - `employeeNumber`: Unique identifier for each employee.
    - `lastName`, `firstName`: Employee's name.
    - `extension`: Employee's phone extension.
    - `email`: Employee's email.
    - `officeCode`: Office where the employee works.
    - `reportsTo`: Employee's manager.
    - `jobTitle`: Employee's job title.

---

### 3. **Table**: `offices`
**Description**: Stores details of the company's offices, such as location and contact info.  
**Primary Key**: `officeCode`
  - **Columns**:
    - `officeCode`: Unique identifier for each office.
    - `city`: City where the office is located.
    - `phone`: Office contact number.
    - `addressLine1`, `addressLine2`: Office address.
    - `state`, `postalCode`, `country`: Location details.
    - `territory`: Geographical area the office covers.

---

### 4. **Table**: `orders`
**Description**: Stores customer orders, including order dates, status, and customer who placed the order.  
**Primary Key**: `orderNumber`  
**Foreign Key**: `customerNumber` → `customers(customerNumber)`
  - **Columns**:
    - `orderNumber`: Unique identifier for each order.
    - `orderDate`: Date the order was placed.
    - `requiredDate`: Deadline for the order.
    - `shippedDate`: Shipment date.
    - `status`: Current order status.
    - `comments`: Additional order comments.
    - `customerNumber`: Customer who placed the order.

---

### 5. **Table**: `orderdetails`
**Description**: Contains line items for each order, detailing the product, quantity, and price.  
**Primary Key**: (`orderNumber`, `productCode`)  
**Foreign Key**: `orderNumber` → `orders(orderNumber)`, `productCode` → `products(productCode)`
  - **Columns**:
    - `orderNumber`: Reference to the order.
    - `productCode`: Reference to the product being ordered.
    - `quantityOrdered`: Number of units ordered.
    - `priceEach`: Price per unit.
    - `orderLineNumber`: Line number within the order.

---

### 6. **Table**: `payments`
**Description**: Records customer payments, including the payment amount and date.  
**Primary Key**: (`customerNumber`, `checkNumber`)  
**Foreign Key**: `customerNumber` → `customers(customerNumber)`
  - **Columns**:
    - `customerNumber`: Reference to the customer.
    - `checkNumber`: Payment identifier.
    - `paymentDate`: Date of payment.
    - `amount`: Amount paid.

---

### 7. **Table**: `products`
**Description**: Contains product information, including pricing, stock levels, and descriptions.  
**Primary Key**: `productCode`  
**Foreign Key**: `productLine` → `productlines(productLine)`
  - **Columns**:
    - `productCode`: Unique identifier for each product.
    - `productName`: Product's name.
    - `productLine`: Product category.
    - `productScale`: Scale of the product (e.g., 1:18).
    - `productVendor`: Vendor supplying the product.
    - `productDescription`: Detailed description.
    - `quantityInStock`: Stock availability.
    - `buyPrice`: Cost to purchase the product.
    - `MSRP`: Manufacturer's suggested retail price.

---

### 8. **Table**: `productlines`
**Description**: Stores product categories, including descriptions and images.  
**Primary Key**: `productLine`
  - **Columns**:
    - `productLine`: Unique identifier for each category.
    - `textDescription`: Brief description.
    - `htmlDescription`: HTML-based content.
    - `image`: Image associated with the product line.

## Loading the Database and Exploring its tables

In [6]:
%load_ext sql
%sql sqlite:///stores.db

In [7]:
%%sql

-- Count columns and rows for each table using UNION ALL

-- Customers table
SELECT 'Customers' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('customers')) AS num_attributes, 
       (SELECT COUNT(*) FROM customers) AS num_rows

UNION ALL

-- Employees table
SELECT 'Employees' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('employees')) AS num_attributes, 
       (SELECT COUNT(*) FROM employees) AS num_rows

UNION ALL

-- Offices table
SELECT 'Offices' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('offices')) AS num_attributes, 
       (SELECT COUNT(*) FROM offices) AS num_rows

UNION ALL

-- Orders table
SELECT 'Orders' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('orders')) AS num_attributes, 
       (SELECT COUNT(*) FROM orders) AS num_rows

UNION ALL

-- OrderDetails table
SELECT 'Orderdetails' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('orderdetails')) AS num_attributes, 
       (SELECT COUNT(*) FROM orderdetails) AS num_rows

UNION ALL

-- Payments table
SELECT 'Payments' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('payments')) AS num_attributes, 
       (SELECT COUNT(*) FROM payments) AS num_rows

UNION ALL

-- Products table
SELECT 'Products' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('products')) AS num_attributes, 
       (SELECT COUNT(*) FROM products) AS num_rows

UNION ALL

-- ProductLines table
SELECT 'Productlines' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('productlines')) AS num_attributes, 
       (SELECT COUNT(*) FROM productlines) AS num_rows;

 * sqlite:///stores.db
Done.


table_name,num_attributes,num_rows
Customers,13,122
Employees,8,23
Offices,9,7
Orders,7,326
Orderdetails,5,2996
Payments,4,273
Products,9,110
Productlines,4,7


## Analysis of Product Stock Levels

In [9]:
%%sql

-- Question 1: Which Products Should We Order More of or Less of?

-- Define the CTEs for low stock and product performance
WITH 
LowStock AS (
    -- CTE to calculate low stock ratio
    SELECT p.productCode, 
           ROUND(SUM(od.quantityOrdered) / p.quantityInStock, 2) AS low_stock_ratioQuestion 2: How Should We Match Marketing and Communication Strategies to Customer Behavior?
      FROM products p
      JOIN orderdetails od ON p.productCode = od.productCode
     GROUP BY p.productCode
    HAVING p.quantityInStock > 0
), 
ProductPerformance AS (
    -- CTE to calculate product performance
    SELECT od.productCode, 
           SUM(od.quantityOrdered * od.priceEach) AS product_performance
      FROM orderdetails od
     GROUP BY od.productCode
)

-- Final query to get priority products for restocking
SELECT p.productCode, 
       p.productName,
       ls.low_stock_ratio, 
       ROUND(pp.product_performance, 2) AS product_performance
  FROM products p
  JOIN LowStock ls ON p.productCode = ls.productCode
  JOIN ProductPerformance pp ON p.productCode = pp.productCode
 ORDER BY ls.low_stock_ratio DESC, pp.product_performance DESC
 LIMIT 10;

 * sqlite:///stores.db
(sqlite3.OperationalError) near "2": syntax error
[SQL: -- Question 1: Which Products Should We Order More of or Less of?

-- Define the CTEs for low stock and product performance
WITH 
LowStock AS (
    -- CTE to calculate low stock ratio
    SELECT p.productCode, 
           ROUND(SUM(od.quantityOrdered) / p.quantityInStock, 2) AS low_stock_ratioQuestion 2: How Should We Match Marketing and Communication Strategies to Customer Behavior?
      FROM products p
      JOIN orderdetails od ON p.productCode = od.productCode
     GROUP BY p.productCode
    HAVING p.quantityInStock > 0
), 
ProductPerformance AS (
    -- CTE to calculate product performance
    SELECT od.productCode, 
           SUM(od.quantityOrdered * od.priceEach) AS product_performance
      FROM orderdetails od
     GROUP BY od.productCode
)

-- Final query to get priority products for restocking
SELECT p.productCode, 
       p.productName,
       ls.low_stock_ratio, 
       ROUND(pp.product_perfor

## Question 1: Which Products Should We Order More of or Less of?

To determine which products to order more of or less of, we conducted an analysis using inventory data to calculate a **low stock ratio** and **product performance** for each product. 

### Interpretation of Results

1. **Low Stock Ratio**:
   - The **low stock ratio** indicates how many times the quantity ordered exceeds the quantity currently in stock.
   - For example, the **1960 BSA Gold Star DBD34** has a low stock ratio of **67.0**, meaning it has been ordered significantly more than what is currently in stock. This indicates high demand and suggests that more units should be ordered to meet customer needs.

2. **Product Performance**:
   - The **product performance** metric is the total revenue generated from sales of each product.
   - The **1968 Ford Mustang**, despite having a lower low stock ratio of **13.0**, has the highest product performance of **161531.48**, highlighting its popularity and revenue-generating potential.

### Summary

Based on the analysis, the products with high low stock ratios and strong performance metrics should be prioritized for restocking. 

- **Recommended to Order More**:
  - **1960 BSA Gold Star DBD34**: Very high demand and significant sales revenue.
  - **1968 Ford Mustang**: A well-performing product that should be kept in stock to meet ongoing demand.

- **Consider Ordering Less**:
  - Products with a low low stock ratio (1.0) may indicate lower demand relative to stock levels. Consider analyzing their sales trends to decide on future orders.

This analysis provides actionable insights into inventory management and helps optimize stock levels for improved sales performance.

## Analysis of Customer Profit and Marketing Strategy

In [13]:
%%sql

WITH CustomerProfit AS (
    SELECT o.customerNumber,
           ROUND(SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)), 2) AS profit
      FROM orders o
      JOIN orderdetails od ON o.orderNumber = od.orderNumber
      JOIN products p ON od.productCode = p.productCode
     GROUP BY o.customerNumber
)

SELECT c.contactLastName, 
       c.contactFirstName, 
       c.city, 
       c.country, 
       cp.profit
  FROM CustomerProfit cp
  JOIN customers c ON cp.customerNumber = c.customerNumber
 ORDER BY cp.profit DESC
 LIMIT 5;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,profit
Freyre,Diego,Madrid,Spain,326519.66
Nelson,Susan,San Rafael,USA,236769.39
Young,Jeff,NYC,USA,72370.09
Ferguson,Peter,Melbourne,Australia,70311.07
Labrune,Janine,Nantes,France,60875.3


In [14]:
%%sql

WITH CustomerProfit AS (
    SELECT o.customerNumber,
           ROUND(SUM(od.quantityOrdered * (od.priQuestion 3: How Much Can We Spend on Acquiring New Customers?ceEach - p.buyPrice)), 2) AS profit
      FROM orders o
      JOIN orderdetails od ON o.orderNumber = od.orderNumber
      JOIN products p ON od.productCode = p.productCode
     GROUP BY o.customerNumber
)
SELECT c.contactLastName, 
       c.contactFirstName, 
       c.city, 
       c.country, 
       cp.profit
  FROM CustomerProfit cp
  JOIN customers c ON cp.customerNumber = c.customerNumber
 ORDER BY cp.profit ASC
 LIMIT 5;

 * sqlite:///stores.db
(sqlite3.OperationalError) near "3": syntax error
[SQL: WITH CustomerProfit AS (
    SELECT o.customerNumber,
           ROUND(SUM(od.quantityOrdered * (od.priQuestion 3: How Much Can We Spend on Acquiring New Customers?ceEach - p.buyPrice)), 2) AS profit
      FROM orders o
      JOIN orderdetails od ON o.orderNumber = od.orderNumber
      JOIN products p ON od.productCode = p.productCode
     GROUP BY o.customerNumber
)
SELECT c.contactLastName, 
       c.contactFirstName, 
       c.city, 
       c.country, 
       cp.profit
  FROM CustomerProfit cp
  JOIN customers c ON cp.customerNumber = c.customerNumber
 ORDER BY cp.profit ASC
 LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Question 2: How Should We Match Marketing and Communication Strategies to Customer Behavior?

Based on our results, we have identified two customer groups: **VIP customers** and **least-engaged customers**. 

### VIP Customers:
These are the top five customers who have generated the most profit for the store. These customers, particularly those like Diego Freyre and Susan Nelson, are extremely valuable to the business. They generate high profit margins and may benefit from personalized, high-touch marketing efforts. Strategies like **exclusive VIP events, loyalty programs**, or **personalized offers** can further increase their engagement and maintain their loyalty to the store.

### Least-Engaged Customers:
On the other hand, we also identified the least-engaged customers who have generated the lowest profit for the store. These customers generate lower profit and are considered less engaged. To improve their engagement, the store can implement **targeted marketing campaigns** aimed at re-engaging these customers. For instance, **discount offers, promotional campaigns**, or **loyalty rewards** can encourage repeat purchases and increase their lifetime value to the store.

### Conclusion:
By understanding these two groups, the store can tailor its marketing and communication strategies to each. For **VIP customers**, personalized and exclusive offers may help maintain and increase their loyalty, while for **less-engaged customers**, promotional incentives could help drive more frequent purchases and boost overall engagement.

## Analysis of Customer Lifetime Value

In [18]:
%%sql

-- CTE to calculate profit for each customer
WITH 
CustomerProfit AS (
    SELECT o.customerNumber,
           ROUND(SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)), 2) AS profit
      FROM orders o
      JOIN orderdetails od ON o.orderNumber = od.orderNumber
      JOIN products p ON od.productCode = p.productCode
     GROUP BY o.customerNumber
)

-- Query to calculate the average profit (Customer Lifetime Value)
SELECT ROUND(AVG(profit), 2) AS customer_lifetime_value
  FROM CustomerProfit;

 * sqlite:///stores.db
Done.


customer_lifetime_value
39039.59


## Question 3: How Much Can We Spend on Acquiring New Customers?

To determine how much we can spend on acquiring new customers, we computed the **Customer Lifetime Value (LTV)**. LTV represents the average profit generated by each customer over the duration of their relationship with the store. Based on our analysis, the average LTV is **$39,039.59**.

This figure is a key indicator of the value each customer brings to the store. Knowing this, the company can make more informed decisions about how much it can invest in acquiring new customers while still maintaining profitability. The amount spent on customer acquisition should ideally be a fraction of the LTV to ensure a positive return on investment.

### Analysis

From our previous results, we observed that the number of new customers has significantly decreased, especially since 2003, and by 2004 the new customer growth rate dropped sharply. By **September 2004**, the store ceased acquiring any new customers, which raises concern about customer retention and acquisition strategies.

Given the high LTV of **$39,039.59**, it is evident that acquiring new customers is crucial, as each customer generates a substantial amount of profit. Based on this LTV, the store can invest in marketing strategies designed to attract new customers, with the goal of maximizing the return on the investment. For instance, marketing campaigns that cost less than a significant portion of the LTV (e.g., 10-20%) would still result in a net gain for the company.

## Conclusion

In this analysis, we addressed three key questions regarding the store's products and customers. First, we identified products that should be prioritized for restocking based on their **low stock ratio** and **sales performance**, ensuring the store stays well-stocked on high-demand items like the **1960 BSA Gold Star DBD34** and **1968 Ford Mustang**. Second, we categorized customers into **VIP** and **less-engaged** groups based on the profit they generate, highlighting opportunities to strengthen relationships with VIPs and engage less active customers.

Finally, we analyzed the store's **customer acquisition** trend and computed the **Customer Lifetime Value (LTV)** of $39,039.59, which provides a solid benchmark for how much the store can spend on acquiring new customers. Given the decline in new customer acquisition, it's clear that investing in targeted marketing strategies will be crucial to reversing this trend and sustaining future growth.